# Advent of Code 2021

## Day 0 : Imports and Utility Functions

In [15]:
import copy
import numpy as np
import re
from collections import defaultdict

In [16]:
def file_to_list(filename, sep="\n", maxsplit=-1) -> list:
    """
    Read an input file and split it using sep as the delimiter.
    """
    with open(filename) as f:
        return f.read().rstrip().split(sep, maxsplit=maxsplit)

## Day 1: Sonar Sweep

### Part 1

Given a sonar report, count the number of times a depth measurement increases from the previous measurement. For example, in the following report, there are 7 measurements that are larger than the previous measurement.

In [3]:
test_report = [199, 200, 208, 210, 200, 207, 240, 269, 260, 263]

In [4]:
def day1_part1(report: list[int]):
    n = 0
    for i in reversed(range(1, len(report))):
        if report[i] > report[i-1]:
            n += 1
    return n

day1_part1(test_report)

7

In [5]:
final_report = [int(value) for value in file_to_list("inputs/input1.txt")]
day1_part1(final_report)

1215

### Part 2

Consider sums of a three-measurement sliding window. How many sums are larger than the previous sum?

In [6]:
# Number of sliding windows
len(test_report) - 2

8

In [7]:
def day1_part2(report: list[int]):
    n = 0
    for i in reversed(range(3, len(report))):
        if sum(report[i-2:i+1]) > sum(report[i-3:i]):
            n += 1
    return n

day1_part2(test_report)

5

In [8]:
day1_part2(final_report)

1150

## Day 2: Dive!

### Part 1

The submarine has a planned course consisting of a list of commands such as `forward 1`, `down 2`, or `up 3`. The horizontal position and depth both start at `0`. Calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?

In [9]:
test_course = ["forward 5", "down 5", "forward 8", "up 3", "down 8", "forward 2"]

In [10]:
def day2_part1(course: list[str]):
    # x: horizontal position, y: depth
    x = y = 0
    for cmd in course:
        d = int(cmd[-1])
        if cmd[0] == "f":
            x += d
        elif cmd[0] == "d":
            y += d
        else:
            y -= d
    return x * y

day2_part1(test_course)

150

In [11]:
final_course = file_to_list("inputs/input2.txt")
day2_part1(final_course)

2150351

### Part 2

New interpretation of the commands:

- `down X` increases your aim by X units.
- `up X` decreases your aim by X units.
- `forward X` does two things:
    - It increases your horizontal position by X units.
    - It increases your depth by your aim multiplied by X.

Using this new interpretation of the commands, calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?

In [12]:
def day2_part2(course: list[str]):
    # x: horizontal position, y: depth
    x = y = aim = 0
    for cmd in course:
        d = int(cmd[-1])
        if cmd[0] == "f":
            x += d
            y += aim * d
        elif cmd[0] == "d":
            aim += d
        else:
            aim -= d
    return x * y

day2_part2(test_course)

900

In [13]:
day2_part2(final_course)

1842742223

## Day 3: Binary Diagnostic

### Part 1

The puzzle input (a diagnostic report) consists of a list of binary numbers. You need to use the binary numbers in the diagnostic report to generate two new binary numbers (called the `gamma rate` and the `epsilon rate`). Each bit in the `gamma rate` can be determined by finding the most common bit in the corresponding position of all numbers in the diagnostic report. The `epsilon rate` is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used.

Use the binary numbers in your diagnostic report to calculate the `gamma rate` and `epsilon rate`, then multiply them together. What is the power consumption of the submarine? (Be sure to represent your answer in decimal, not binary.)

In [14]:
test_report = ["00100", "11110", "10110", "10111", "10101", "01111", "00111", "11100", "10000", "11001", "00010", "01010"]

In [15]:
def day3_part1(report: list[str]):
    gamma = ""
    for i in range(len(report[0])):
        n0 = 0
        for j in range(len(report)):
            if report[j][i] == "0":
                n0 += 1
        if n0 >= len(report) // 2:
            gamma += "0"
        else:
            gamma += "1"
    gamma = int(gamma, 2)
    epsilon = 2 ** len(report[0]) - 1 - gamma
    return gamma * epsilon

day3_part1(test_report)

198

In [16]:
final_report = file_to_list("inputs/input3.txt")
day3_part1(final_report)

841526

### Part 2

Next, consider the `oxygen generator rating` and the `CO2 scrubber rating`. To find `oxygen generator rating`, determine the most common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 1 in the position being considered. To find `CO2 scrubber rating`, determine the least common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 0 in the position being considered.

Use the binary numbers in your diagnostic report to calculate the `oxygen generator rating` and `CO2 scrubber rating`, then multiply them together. What is the life support rating of the submarine? (Be sure to represent your answer in decimal, not binary.)

In [17]:
def day3_part2(report: list[str]):
    oxygen = c02 = report
    for i in range(len(oxygen[0])):
        if len(oxygen) > 1:
            n0 = 0
            for j in range(len(oxygen)):
                if oxygen[j][i] == "0":
                    n0 += 1
            most = "1"
            if n0 > len(oxygen) // 2:
                most = "0"
            oxygen = [o for o in oxygen if o[i] == most]
    for i in range(len(c02[0])):
        if len(c02) > 1:
            n0 = 0
            for j in range(len(c02)):
                if c02[j][i] == "0":
                    n0 += 1
            least = "1"
            if n0 <= len(c02) // 2:
                least = "0"
            c02 = [c for c in c02 if c[i] == least]
    return int(oxygen[0], 2) * int(c02[0], 2)

day3_part2(test_report)

230

In [18]:
day3_part2(final_report)

4790390

## Day 4: Giant Squid

### Part 1

Play bingo with the giant squid that has attached itself to the submarine.

Given a random sequence of numbers and random set of boards, find the winning board. If all numbers in any row or any column of a board are marked, that board wins. Find the sum of all unmarked numbers on the winning board and multiply that sum by the number that was just called when the board won.

In [19]:
test_draw_numbers = [7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1]
test_boards = [[[22, 13, 17, 11, 0],
  [8, 2, 23, 4, 24],
  [21, 9, 14, 16, 7],
  [6, 10, 3, 18, 5],
  [1, 12, 20, 15, 19]],
 [[3, 15, 0, 2, 22],
  [9, 18, 13, 17, 5],
  [19, 8, 7, 25, 23],
  [20, 11, 10, 24, 4],
  [14, 21, 16, 12, 6]],
 [[14, 21, 17, 24, 4],
  [10, 16, 15, 9, 19],
  [18, 8, 23, 26, 20],
  [22, 11, 13, 6, 5],
  [2, 0, 12, 3, 7]]]

In [20]:
def day4_part1(numbers: list[str], boards: list[list]):
    winner = None
    for number in numbers:
        for board in boards:
            # Mark the new number
            for i in range(5):
                h = v = 0
                for j in range(5):
                    if number == board[i][j]:
                        board[i][j] = -1
                    if board[i][j] == -1:
                        h += 1
                    if board[j][i] == -1:
                        v += 1
                # Check if the board has won
                if h == 5 or v == 5:
                    winner = board
                    break
            if winner:
                score = sum(sum(x for x in row if x != -1) for row in board)
                print(score * number)
                return

day4_part1(copy.deepcopy(test_draw_numbers), copy.deepcopy(test_boards))

4512


In [21]:
input4 = file_to_list("inputs/input4.txt", "\n\n", 1)
final_draw_numbers = [int(n) for n in input4[0].split(",")]
final_boards = [x.split("\n") for x in input4[1].split("\n\n")]
final_boards = [[[int(z) for z in y.split()] for y in x] for x in final_boards]

In [22]:
day4_part1(copy.deepcopy(final_draw_numbers), copy.deepcopy(final_boards))

27027


### Part 2

Let's try a different strategy: let the giant squid win.

Figure out which board will win last. Once it wins, what would its final score be?

In [23]:
def day4_part2(numbers: list[str], boards: list[list]):
    for number in numbers:
        for b in reversed(range(len(boards))):
            won = False
            # Mark the new number
            for i in range(5):
                h = v = 0
                for j in range(5):
                    if number == boards[b][i][j]:
                        boards[b][i][j] = -1
                    if boards[b][i][j] == -1:
                        h += 1
                    if boards[b][j][i] == -1:
                        v += 1
                # Check if the board has won
                if h == 5 or v == 5:
                    won = True            
            if won == True:
                if len(boards) == 1:
                    score = sum(sum(x for x in row if x != -1) for row in boards[b])
                    print(score * number)
                boards.remove(boards[b])

day4_part2(copy.deepcopy(test_draw_numbers), copy.deepcopy(test_boards))

1924


In [24]:
day4_part2(final_draw_numbers, final_boards)

36975


## Day 5: Hydrothermal Venture

### Part 1

You come across a field of hydrothermal vents on the ocean floor!

Hydrothermal vents tend to form lines that are horizontal, vertical or diagonal (45 degrees). The submarine generates a list of nearby lines of vents.

For the first part, consider only horizontal and vertical lines. At how many points do at least two lines overlap?

In [25]:
test_lines = [[0, 9, 5, 9],
 [8, 0, 0, 8],
 [9, 4, 3, 4],
 [2, 2, 2, 1],
 [7, 0, 7, 4],
 [6, 4, 2, 0],
 [0, 9, 2, 9],
 [3, 4, 1, 4],
 [0, 0, 8, 8],
 [5, 5, 8, 2]]

In [26]:
def day5_part1(coordinates:list[list]):
    # 2-D array
    array = np.zeros((1000, 1000))
    for x1, y1, x2, y2 in coordinates:
        # Horizontal lines (x1 = x2)
        if x1 == x2:
            array[x1, min(y1, y2):max(y1, y2) + 1] += 1
        # Vertical lines (y1 = y2)
        elif y1 == y2:
            array[min(x1, x2):max(x1, x2) + 1, y1] += 1
    return (array > 1).sum()

day5_part1(test_lines)

5

In [27]:
final_lines = file_to_list("inputs/input5.txt")
final_lines = [list(map(int, re.split(",| -> ", line))) for line in final_lines]

In [28]:
day5_part1(final_lines)

5084

### Part 2

For the second part, consider all of the lines. At how many points do at least two lines overlap?

In [29]:
def day5_part2(coordinates:list[list]):
    # 2-D array
    array = np.zeros((1000, 1000))
    for x1, y1, x2, y2 in coordinates:
        # Horizontal lines (x1 = x2)
        if x1 == x2:
            array[x1, min(y1, y2):max(y1, y2)+1] += 1
        # Vertical lines (y1 = y2)
        elif y1 == y2:
            array[min(x1, x2):max(x1, x2)+1, y1] += 1
        # Diagonal lines
        else:
            # range() evaluates to False if the sequence is not continuous
            xrange = range(x1, x2 + 1) or range(x1, x2 - 1, -1)
            yrange = range(y1, y2 + 1) or range(y1, y2 - 1, -1)
            for x, y in zip(xrange, yrange):
                array[x, y] += 1
    return (array > 1).sum()

day5_part2(test_lines)


12

In [30]:
day5_part2(final_lines)

17882

## Day 6: Lanternfish

### Part 1

Model the rate of growth of a school of lanternfish.

Each fish can be modelled by a single number that represents the number of days until it creates a new lanternfish Each lanternfish creates a new lanternfish once every 7 days. A new lanternfish needs 2 more days for its first cycle.

Find a way to simulate lanternfish. How many lanternfish would there be after 80 days?

In [31]:
# 0 counts as a valid timer value
test_fishes = [3,4,3,1,2]

In [32]:
def day6_part1(fishes: list[int], days: int):
    for _ in range(days):
        new = []
        for idx, fish in enumerate(fishes):
            if fish == 0:
                new.append(8)
                fishes[idx] = 6
            else:
                fishes[idx] -= 1
        fishes = fishes + new
    return len(fishes)

day6_part1(copy.deepcopy(test_fishes), 80)

5934

In [33]:
final_fishes = [int(fish) for fish in file_to_list("inputs/input6.txt", ",")]

In [34]:
day6_part1(copy.deepcopy(final_fishes), 80)

359999

### Part 2

Suppose the lanternfish live forever and have unlimited food and space. Would they take over the entire ocean?

How many lanternfish would there be after 256 days?

In [35]:
# The growth rate of lanternfish is exponential.
# Our solution to part 1 has therefore a space and time complexity of O(n^2),
# and is uncomputable with large parameters (e.g. days=256).

In [36]:
def day6_part2(fishes: list[int], days:int):
    """
    Lanternfish growth model with a better space and time complexity.
    """
    state = [fishes.count(i) for i in range(9)]
    for _ in range(days):
        state = state[1:] + state[:1]
        state[6] += state[-1]
    return sum(state)

day6_part2(copy.deepcopy(test_fishes), 256)

26984457539

In [37]:
day6_part2(copy.deepcopy(final_fishes), 256)

1631647919273

## Day 7: The Treachery of Whales

### Part 1

A giant whale is pursuing our submarine, but an army of crabs in their mini-submarines is coming to our rescue! By aligning on a same line, the crabs could blast a hole in the ocean floor and give us access to an underground cave system where we could take shelter from the whale.

Crab submarines can only move horizontally. Each change of 1 step in horizontal position of a single crab costs 1 fuel. The puzzle input is the original horizontal positions of the crabs.

Determine the horizontal position that the crabs can align to using the least fuel possible. How much fuel must they spend to align to that position?

In [38]:
test_positions = [16,1,2,0,4,2,7,1,2,14]

In [39]:
def day7_part1(positions: list[int]):
    median = np.median(positions)
    return int(sum(abs(positions - median)))

day7_part1(test_positions)

37

In [40]:
final_positions = [int(p) for p in file_to_list("inputs/input7.txt", sep=",")]

In [41]:
day7_part1(final_positions)

352331

### Part 2

As it turns out, crab submarine engines don't burn fuel at a constant rate. Instead, each change of 1 step in horizontal position costs 1 more unit of fuel than the last: the first step costs 1, the second step costs 2, the third step costs 3, and so on.

Determine the horizontal position that the crabs can align to using the least fuel possible so they can make you an escape route! How much fuel must they spend to align to that position?

In [42]:
def day7_part2(positions: list[int]):
    """
    The optimal position is within 0,5 from the mean,
    so we take the mean, calculate the sum of the fuel costs
    around the mean, and take whichever value is lower. 
    The fuel cost for each submarine is calculated by
    substracting the mean from its position and calculating
    the sum of the arithmetic sequence using Gauss' method.
    E.g. for position 16 if mean = 5:
    16 - 5 = 11
    cost = 11 * (11 + 1) // 2 = 66
    """
    mean = np.mean(positions)
    cost = lambda x: x * (x+1) // 2
    c = sum(cost(abs(positions-np.ceil(mean))))
    f = sum(cost(abs(positions-np.floor(mean))))
    return int(min(c,f))

day7_part2(test_positions)

168

In [43]:
day7_part2(final_positions)

99266250

## Day 8: Seven Segment Search

## Part 1

The [seven-segment displays](https://en.wikipedia.org/wiki/Seven-segment_display) in the submarine are malfunctioning. 

Each digit of a seven-segment display is rendered by turning on or off any of seven segments named a through g:

```python
 0:      1:      2:      3:      4:      5:      6:      7:      8:      9:
 aaaa    ....    aaaa    aaaa    ....     aaaa    aaaa    aaaa    aaaa    aaaa
b    c  .    c  .    c  .    c  b    c   b    .  b    .  .    c  b    c  b    c
b    c  .    c  .    c  .    c  b    c   b    .  b    .  .    c  b    c  b    c
 ....    ....    dddd    dddd    dddd     dddd    dddd    ....    dddd    dddd
e    f  .    f  e    .  .    f  .    f   .    f  e    f  .    f  e    f  .    f
e    f  .    f  e    .  .    f  .    f   .    f  e    f  .    f  e    f  .    f
 gggg    ....    gggg    gggg    ....     gggg    gggg    ....    gggg    gggg
```

The puzzle input is a series on entries. Each entry consists of ten unique signal patterns, a | delimiter, and finally the four digit output value. Because the digits 1, 4, 7, and 8 each use a unique number of segments, you should be able to tell which combinations of signals correspond to those digits.

In the output values, how many times do digits 1, 4, 7, or 8 appear?

In [44]:
test_segments = [
    "be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe",
    "edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc",
    "fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg",
    "fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb",
    "aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea",
    "fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb",
    "dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe",
    "bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef",
    "egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb",
    "gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce",
]

In [45]:
def day8_part1(segments: list[str]):
    s = [seg.split(" | ")[1] for seg in segments]
    s = [seg.split() for seg in s]
    count = 0
    for entry in s:
        for digit in entry:
            if len(digit) in (2,3,4,7):
                count += 1
    return count

day8_part1(test_segments)

26

In [46]:
final_segments = file_to_list("inputs/input8.txt")

In [47]:
day8_part1(final_segments)

349

### Part 2

Through a little deduction, you should now be able to determine the remaining digits.

For each entry, determine all of the wire/segment connections and decode the four-digit output values. What do you get if you add up all of the output values?

In [48]:
def day8_part2(segments: list[str]):
    sum = 0
    entries = [tuple(x.split() for x in seg.split(" | ")) for seg in segments]
    for patterns, outputs in entries:
        value = ""
        # Map the digits with a unique lenght
        known = {}
        for pattern in patterns:
            match len(pattern):
                case 2: known["1"] = pattern
                case 3: known["7"] = pattern
                case 4: known["4"] = pattern
                case 7: known["8"] = pattern
        # Deduce the rest of the digits based on
        # the number of common segments with 1 and 4
        map = {}
        for pattern in patterns:
            sig = "".join(sorted(pattern))
            match len(sig), len(set(sig).intersection(known["1"])), len(set(sig).intersection(known["4"])):
                case 2, _, _ : map[sig] = "1"
                case 3, _, _ : map[sig] = "7"
                case 4, _, _ : map[sig] = "4"
                case 7, _, _ : map[sig] = "8"
                case 5, 1, 2 : map[sig] = "2"
                case 5, 1, 3 : map[sig] = "5"
                case 5, 2, 3 : map[sig] = "3"
                case 6, _, 4 : map[sig] = "9"
                case 6, 1, 3 : map[sig] = "6"
                case 6, 2, 3 : map[sig] = "0"
        for output in outputs:
            digit = map.get("".join(sorted(output)))
            value += digit
        sum += int(value)
    return sum

day8_part2(test_segments)

61229

In [49]:
day8_part2(final_segments)

1070957

## Day 9: Smoke Basin

### Part 1

Smoke flows through the cave system. It flows to the lowest point of the area it's in.

Low points are defined as the locations that are lower than any of their adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)

The risk level of a low point is 1 plus its height.

Find all of the low points on your heightmap. What is the sum of the risk levels of all low points on your heightmap?

In [50]:
# Heightmap: each point corresponds to the height of a particular location
test_grid = [
    [2,1,9,9,9,4,3,2,1,0],
    [3,9,8,7,8,9,4,9,2,1],
    [9,8,5,6,7,8,9,8,9,2],
    [8,7,6,7,8,9,6,7,8,9],
    [9,8,9,9,9,6,5,6,7,8]
]

In [51]:
def day9_part1(grid: list[list]):
    sum = 0
    for x in range(len(grid)):
        for y in range(len(grid[0])):
            lowest = True
            coordinates = [(-1, 0), (0, 1), (1, 0), (0, -1)]
            for dx, dy in coordinates:
                if 0 <= x+dx < len(grid) and 0 <= y+dy < len(grid[0]):
                    if grid[x+dx][y+dy] <= grid[x][y]:
                        lowest = False
                if not lowest:
                    break
            if lowest:
                sum += grid[x][y] + 1
    return sum

day9_part1(test_grid)

15

In [52]:
final_grid = [list(map(int, list(line))) for line in file_to_list("inputs/input9.txt")]

In [53]:
day9_part1(final_grid)

480

### Part 2

A basin is all locations that eventually flow downward to a single low point. Therefore, every low point has a basin, although some basins are very small. Locations of height 9 do not count as being in any basin, and all other locations will always be part of exactly one basin.

The size of a basin is the number of locations within the basin, including the low point.

What do you get if you multiply together the sizes of the three largest basins?

In [54]:
def bfs(grid: list[list], point: tuple):
    """Breadth-first search"""
    queue = [point]
    visited = set()
    while queue:
        x, y = queue.pop(0)
        coordinates = [(-1, 0), (0, 1), (1, 0), (0, -1)]
        for dx, dy in coordinates:
            if 0 <= x+dx < len(grid) and 0 <= y+dy < len(grid[0]):
                n = (x+dx, y+dy)
                # Locations of height 9 mark the bassins' borders
                if n in visited or grid[x+dx][y+dy] == 9:
                    continue
                visited.add(n)
                queue.append(n)
    return len(visited)

def day9_part2(grid: list[list]):
    basins = []
    for x in range(len(grid)):
        for y in range(len(grid[0])):
            lowest = True
            coordinates = [(-1, 0), (0, 1), (1, 0), (0, -1)]
            for dx, dy in coordinates:
                if 0 <= x+dx < len(grid) and 0 <= y+dy < len(grid[0]):
                    if grid[x+dx][y+dy] <= grid[x][y]:
                        lowest = False
                if not lowest:
                    break
            if lowest:
                bassin_size = bfs(grid, (x, y))
                basins.append(bassin_size)
    basins = sorted(basins)[-3:]
    return basins[0] * basins[1] * basins[2]

day9_part2(test_grid)

1134

In [55]:
day9_part2(final_grid)

1045660

## Day 10: Syntax Scoring

### Part 1

The puzzle input is a list of code lines: `[<>({}){}[([])<>]]`.

Each line is made up of chunks which are opened and closed with the following characters: `[]`, `{}`, `<>`, `()`.

Some lines are incomplete, but others are corrupted. A corrupted line is one where a chunk closes with the wrong character. 

The syntax checker scores syntax errors based on the first illegal character in each line. Illegal characters give the following number of points:

- `)`: 3 points
- `]`: 57 points
- `}`: 1197 points
- `>`: 25137 points

Find the first illegal character in each corrupted line of the navigation subsystem. What is the total syntax error score for those errors?

In [56]:
test_lines = [
    "[({(<(())[]>[[{[]{<()<>>",
    "[(()[<>])]({[<{<<[]>>(",
    "{([(<{}[<>[]}>{[]{[(<()>",
    "(((({<>}<{<{<>}{[]{[]{}",
    "[[<[([]))<([[{}[[()]]]",
    "[{[{({}]{}}([{[{{{}}([]",
    "{<[[]]>}<{[{[{[]{()[[[]",
    "[<(<(<(<{}))><([]([]()",
    "<{([([[(<>()){}]>(<<{{",
    "<{([{{}}[<[[[<>{}]]]>[]]"
]

In [57]:
def day10_part1(lines: list[str]):
    score = 0
    points = {")": 3, "]": 57, "}": 1197, ">": 25137}
    pairs = {"(": ")", "[": "]", "{": "}", "<": ">"}
    for line in lines:
        stack = []
        for char in line:
            if char in ["}", ")", "]", ">"]:
                if char != stack[-1]:
                    score += points[char]
                    break
                else:
                    stack.pop(-1)
            else:
                stack.append(pairs[char])
    return score

day10_part1(test_lines)

26397

In [58]:
final_lines = file_to_list("inputs/input10.txt")

In [59]:
day10_part1(final_lines)

311949

### Part 2

Discard the corrupted lines. Now, only the incomplete lines remain.

Incomplete lines don't have any incorrect characters - instead, they're missing some closing characters at the end of the line.

Find the sequence of closing characters that complete the incomplete lines and score each completion string. The score for each completion string is calculated character-by-character. For each character, multiply the total score by 5 and increase the total score by the point value given for the character:

- `)`: 1 points
- `]`: 2 points
- `}`: 3 points
- `>`: 4 points

Sort the scores for all completion string and return the middle score (there is always an odd number of scores).

In [60]:
def day10_part2(lines: list[str]):
    scores = []
    points = {")": 1, "]": 2, "}": 3, ">": 4}
    pairs = {"(": ")", "[": "]", "{": "}", "<": ">"}
    for line in lines:
        stack = []
        corrupt = False
        for char in line:
            if char in ["}", ")", "]", ">"]:
                if char != stack[-1]:
                    corrupt = True
                else:
                    stack.pop(-1)
            else:
                stack.append(pairs[char])
        if not corrupt:
            score = 0
            for char in reversed(stack):
                score = score * 5 + points[char]
            scores.append(score)
    scores = sorted(scores)
    return scores[len(scores)//2]

day10_part2(test_lines)

288957

In [61]:
day10_part2(final_lines)

3042730309

## Day 11: Dumbo Octopus

### Part 1

100 bioluminescent octopuses are arranged in a 10 by 10 grid.

Each octopus has an energy level that increases over time and is is represented by a value between 0 and 9. Any octopus with an energy level greater than 9 **flashes**. The energy levels and flashes of the group of octopuses can be modelled in steps comprised of three stages:

1. Increase the energy level of each octopus by 1.
2. Any octopus with an energy level greater than 9 flashes. This increases the energy level of all adjacent octopuses by 1. If this causes an octopus to have an energy level greater than 9, it also flashes. This process continues as long as new octopuses keep having their energy level increased beyond 9. (An octopus can only flash at most once per step.)
3. Any octopus that flashed during this step has its energy level set to 0.

Given the starting energy levels of the dumbo octopuses in your cavern, simulate 100 steps. How many total flashes are there after 100 steps?

In [62]:
test_octopuses = np.array([
    [5,4,8,3,1,4,3,2,2,3],
    [2,7,4,5,8,5,4,7,1,1],
    [5,2,6,4,5,5,6,1,7,3],
    [6,1,4,1,3,3,6,1,4,6],
    [6,3,5,7,3,8,5,4,7,8],
    [4,1,6,7,5,2,4,6,4,5],
    [2,1,7,6,8,4,1,7,2,1],
    [6,8,8,2,8,8,1,1,3,4],
    [4,8,4,6,8,4,8,5,5,4],
    [5,2,8,3,7,5,1,5,2,6]
])

In [63]:
def flash(octopuses: np.array, x: int, y: int):
    octopuses[x][y] = 0
    flashes = 1
    for dx in [-1,0,1]:
        for dy in [-1,0,1]:
            nx = x + dx
            ny = y + dy
            if 0 <= nx < 10 and 0 <= ny < 10 and octopuses[nx][ny] != 0:
                octopuses[nx][ny] += 1
                if octopuses[nx][ny] >= 10:
                    flashes += flash(octopuses, nx, ny)
    return flashes

def day11_part1(octopuses: np.array):
    flashes = 0
    for _ in range(100):
        octopuses += 1
        for x in range(10):
            for y in range(10):
                if octopuses[x][y] > 9:
                    flashes += flash(octopuses, x, y)
    return flashes

day11_part1(test_octopuses.copy())

1656

In [64]:
final_octopuses = np.array([list(map(int, row)) for row in file_to_list("inputs/input11.txt")])

In [65]:
day11_part1(final_octopuses.copy())

1729

### Part 2

What is the first step during which all octopuses flash simultaneously?

In [66]:
def day11_part2(octopuses: np.array):
    synchronous = False
    step = 0
    while not synchronous:
        flashes = 0
        octopuses += 1
        for x in range(10):
            for y in range(10):
                if octopuses[x][y] > 9:
                    flashes += flash(octopuses, x, y)
        if flashes == 100:
            synchronous = True
        step += 1
    return step

day11_part2(test_octopuses.copy())

195

In [67]:
day11_part2(final_octopuses.copy())

237

## Day 12: Passage Passing

### Part 1

The puzzle input is a map of a cave system with one `start` and `end` caves. The map indicates how all the caves are connected. There are two types of caves: big (uppercase) and small caves (lowercase). Small caves can be visited at most once, while big caves can be visited any number of times.

How many paths through this cave system are there that visit small caves at most once?

In [23]:
# Represent map as an adjencency list
test_map = defaultdict(
    list, {
        "start": ["A", "b"],
        "A": ["start", "c", "b", "end"],
        "b": ["start", "A", "d", "end"],
        "c": ["A"],
        "d": ["b"],
        "end": ["A", "b"]
    }
)

In [52]:
def day12_part1(map: defaultdict, cave: str, visited: set):
    """Recursive depth-first search"""
    paths = 0
    if cave == "end":
        return 1
    if cave in visited and cave.islower():
        return 0
    for adjacent in map[cave]:
        paths += day12_part1(map, adjacent, visited | {cave})
    return paths

day12_part1(test_map, "start", set())

10

In [25]:
final_map = defaultdict(list)
for line in file_to_list("inputs/input12.txt"):
    a, b = line.split('-')
    final_map[a] += [b]
    final_map[b] += [a]

In [26]:
day12_part1(final_map, "start", set())

4659

### Part 2

New rules: one small cave can be visited twice, while the `start` and `end` caves can be visited only once.

How many paths are there through this cave system?

In [51]:
def day12_part2(map: defaultdict, cave: str, visited: set, twice: str):
    paths = 0
    if cave == "end":
        return 1
    if cave in visited:
        if cave == "start":
            return 0
        if cave.islower():
            if twice is None:
                twice = cave
            else:
                return 0
    for adjacent in map[cave]:
        paths += day12_part2(map, adjacent, visited | {cave}, twice)
    return paths

day12_part2(test_map, "start", set(), None)

36

In [48]:
day12_part2(final_map, "start", set(), None)

148962